# Rock Paper Scissors - RNG Statistics

In theory, the Nash Equlibrium Random Bot cannot be beaten, and will statistically draw against any opponent.

In practice, the classical computer implemention is a pseudo-random number generator, rather than a truly random quantum random number generator.

So the question is if there exist observable patterns in python's [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) algorithm?

For this notebook, we are going spend the entire notebook runtime generating random number sequences, then use this as a lookup table to weight the randomness of our own random bot. If there are patterns to be found in the Mersenne Twister algorithm, then we should have a statistical edge.

# Data Encoder

Little trick I learnt from [Connect4](https://www.kaggle.com/jamesmcguigan/connectx-mcts-bitboard-bitsquares-heuristic/), this is how you load a data file into a kaggle submission.py file, by base64 encoding it and importing it as an inline string. 

In [ ]:
%%writefile base64_file.py
# Source: https://github.com/JamesMcGuigan/ai-games/blob/master/games/connectx/util/base64_file.py

import base64
import gzip
import os
import re
import time
import yaml
from typing import Any
from typing import Union

import pickle
import dill
import humanize


# _base64_file__test_base64_static_import = """
# H4sIAPx9LF8C/2tgri1k0IjgYGBgKCxNLS7JzM8rZIwtZNLwZvBm8mYEkjAI4jFB2KkRbED1iXnF
# 5alFhczeWqV6AEGfwmBHAAAA
# """


def base64_file_varname(filename: str) -> str:
    # ../data/AntColonyTreeSearchNode.dill.zip.base64 -> _base64_file__AntColonyTreeSearchNode__dill__zip__base64
    varname = re.sub(r'^.*/',   '',   filename)  # remove directories
    varname = re.sub(r'[.\W]+', '__', varname)   # convert dots and non-ascii to __
    varname = f"_base64_file__{varname}"
    return varname


def base64_file_var_wrap(base64_data: Union[str,bytes], varname: str) -> str:
    return f'{varname} = """\n{base64_data.strip()}\n"""'                    # add varname = """\n\n""" wrapper


def base64_file_var_unwrap(base64_data: str) -> str:
    output = base64_data.strip()
    output = re.sub(r'^\w+ = """|"""$', '', output)  # remove varname = """ """ wrapper
    output = output.strip()
    return output


def base64_file_encode(data: Any) -> str:
    encoded = pickle.dumps(data)
    encoded = gzip.compress(encoded)
    encoded = base64.encodebytes(encoded).decode('utf8').strip()
    return encoded


def base64_file_decode(encoded: str) -> Any:
    data = base64.b64decode(encoded)
    data = gzip.decompress(data)
    data = pickle.loads(data)
    return data


def base64_file_save(data: Any, filename: str, vebose=True) -> float:
    """
        Saves a base64 encoded version of data into filename, with a varname wrapper for importing via kaggle_compile.py
        # Doesn't create/update global variable.
        Returns filesize in bytes
    """
    varname    = base64_file_varname(filename)
    start_time = time.perf_counter()
    try:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as file:
            encoded = base64_file_encode(data)
            output  = base64_file_var_wrap(encoded, varname)
            output  = output.encode('utf8')
            file.write(output)
            file.close()
        if varname in globals(): globals()[varname] = encoded  # globals not shared between modules, but update for saftey

        filesize = os.path.getsize(filename)
        if vebose:
            time_taken = time.perf_counter() - start_time
            print(f"base64_file_save(): {filename:40s} | {humanize.naturalsize(filesize)} in {time_taken:4.1f}s")
        return filesize
    except Exception as exception:
        print(f'base64_file_save({filename}): Exception:', exception)
    return 0.0


def base64_file_load(filename: str, vebose=True) -> Union[Any,None]:
    """
        Performs a lookup to see if the global variable for this file alread exists
        If not, reads the base64 encoded file from filename, with an optional varname wrapper
        # Doesn't create/update global variable.
        Returns decoded data
    """
    varname    = base64_file_varname(filename)
    start_time = time.perf_counter()
    try:
        # Hard-coding PyTorch weights into a script - https://www.kaggle.com/c/connectx/discussion/126678
        encoded = None

        if varname in globals():
            encoded = globals()[varname]

        if encoded is None and os.path.exists(filename):
            with open(filename, 'rb') as file:
                encoded = file.read().decode('utf8')
                encoded = base64_file_var_unwrap(encoded)
                # globals()[varname] = encoded  # globals are not shared between modules

        if encoded is not None:
            data = base64_file_decode(encoded)

            if vebose:
                filesize = os.path.getsize(filename)
                time_taken = time.perf_counter() - start_time
                print(f"base64_file_load(): {filename:40s} | {humanize.naturalsize(filesize)} in {time_taken:4.1f}s")
            return data
    except Exception as exception:
        print(f'base64_file_load({filename}): Exception:', exception)
    return None


In [ ]:
%run -i 'base64_file.py'

# RNG Statistics Generator

In [ ]:
%%writefile generator.py

from typing import Any, List, Union
import sys
import os
import time
import random
import pickle
import gzip
import numpy as np
import tensorflow as tf
from humanize import naturalsize, precisedelta, intcomma


# https://github.com/JamesMcGuigan/kaggle-digit-recognizer/blob/master/src/random/random_seed_search.py
def get_randoms(length, seed, method='random') -> Union[List[int],np.ndarray]:
    if method == 'random':
        random.seed(seed)        
        return [ random.randint(0,2) for n in range(length) ]
    if method == 'np':
        np.random.seed(seed)
        return np.random.randint(0,2, length)
    if method == 'tf':
        tf.random.set_seed(seed)
        return tf.random.uniform((length,), minval=0, maxval=3, dtype=tf.dtypes.int32).numpy()
    
    
# NOTE: state space = filesize = 3**window
def generate_random_sequence_table(timeout=8.8*60*60, window=10, numbers_per_seed=1000, method='random', lookup=None):
    time_start = time.perf_counter()
    lookup     = lookup or {}  # pickle breaks if given defaultdict()
    count      = 0
    for seed in range(sys.maxsize):
        if time.perf_counter() - time_start > timeout: break
        numbers = get_randoms(numbers_per_seed, seed=seed, method=method)
        for n in range(len(numbers)-window-1):
            sequence    = tuple(numbers[n:n+window])
            next_number = int(numbers[n+window])                
            if sequence not in lookup: lookup[sequence] = [0,0,0]  
            lookup[sequence][next_number] += 1
            count                         += 1

    time_taken = time.perf_counter() - time_start
    print(f'{intcomma(count)} samples / {intcomma(len(lookup))} sequences' + 
          f' = {count/len(lookup):.1f} samples/sequences' + 
          f' = {100*len(lookup)/(3**window):.0f}% ' +
          f'in {precisedelta(time_taken)}')
    
    return lookup


if __name__ == '__main__':
    timeout = 1*60*60 if os.environ.get('KAGGLE_KERNEL_RUN_TYPE','') != 'Interactive' else 12

    # lookup = base64_file_load('../input/rock-paper-scissors-rng-statistics/random_sequence.base64')
    lookup = generate_random_sequence_table(timeout=timeout)
    base64_file_save(lookup, './random_sequence.base64')        


In [ ]:
%run -i 'generator.py'
!head ./random_sequence.base64

# Agent

In [ ]:
%%writefile agent.py

import time
import os
import random
import pickle
import gzip
from typing import Any
from humanize import naturalsize, precisedelta, intcomma


def read_gzip_pickle_file(filename: str, verbose=True) -> Any:
    time_start = time.perf_counter()
    try:
        if not os.path.exists(filename): raise FileNotFoundError
        with open(filename, 'rb') as file:
            data = file.read()
            try:    data = gzip.decompress(data)
            except: pass
            data = pickle.loads(data)
            time_taken = time.perf_counter() - time_start 
            filesize   = os.path.getsize(filename)
            if verbose: print(f'read: {filename} = {naturalsize(filesize)} in {precisedelta(time_taken)}')
    except Exception as exception:
        print('read_gzip_pickle_file()', exception)
        data = None
    return data


lookup = base64_file_load('./random_sequence.base64')
print(f'lookup = {type(lookup)} len({intcomma(len(lookup))})')
window = len(next(iter(lookup.keys())))

history = []
stats = {
    "hit":   0,
    "miss":  0,
    "total": 0,
}
def rng_statistics_agent(observation, configuration):
    global stats
    if observation.step > 0:
        history.append( observation.lastOpponentAction )
    sequence = tuple(history[-window:])
    
    if len(sequence) == window: stats['total'] += 1 
    if sequence in lookup:
        if len(sequence) == window: stats['hit'] += 1 
        weights = lookup[sequence]
        print(f"{stats['hit']:3d}/{stats['total']:3d} = {100*stats['hit']/stats['total']:3.0f}% | {sequence} = {weights}")
    else:
        if len(sequence) == window: stats['miss'] += 1 
        weights = [1,1,1]

    expected_action = random.choices( population=[0,1,2], weights=weights, k=1 )[0]
    counter_action  = ( expected_action + 1 ) % configuration.signs    
    return counter_action


Rules state agent must read file in less than 61 seconds

In [ ]:
%run -i 'agent.py'

The kaggle runtime submission environment doesn't have access to the filesystem, so import data as an inline base64 string

In [ ]:
!cat ./random_sequence.base64 ./base64_file.py ./agent.py > ./submission.py
!mv ./base64_file.py ./base64_file.original.py

# Play

In [ ]:
from kaggle_environments import make

env = make("rps", configuration={"episodeSteps": 100}, debug=True)
env.run(["submission.py", lambda obs, conf: random.randint(0, 2)])
print(env.render(mode="ipython", width=600, height=600))

# Further Reading

This notebook is part of a series exploring Rock Paper Scissors:
- [Rock Paper Scissors - Random Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-random-agent)
- [Rock Paper Scissors - Weighted Random Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-weighted-random-agent)
- [Rock Paper Scissors - Statistical Prediction](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-statistical-prediction)
- [Rock Paper Scissors - XGBoost](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-xgboost)
- [Rock Paper Scissors - Random Seed Search](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-random-seed-search)
- [Rock Paper Scissors - RNG Statistics](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-rng-statistics)